In [2]:
import pandas as pd
import numpy as np
import sqlalchemy

SQL_CREDENTIALS = "root:1234"
engine = sqlalchemy.create_engine('mysql+pymysql://' + SQL_CREDENTIALS + '@localhost:3306/music_recommender_db')
COL_SEPARATOR = "_123456789987654321_"

In [3]:
def _genres_to_list(string_of_genres):
    if string_of_genres == "NaN":
        return np.nan
    else:
        return string_of_genres.split("\t")[:-1]

In [17]:
lowms_users = pd.read_csv("data/low_main_users.txt", usecols=["user_id"], squeeze=True).tolist()
lowms_users

[1049656,
 1055118,
 1056935,
 1070023,
 1072752,
 2043627,
 2052756,
 2064055,
 2073140,
 2095434,
 2109450,
 2164032,
 2246769,
 2246867,
 2248678,
 2271579,
 2324805,
 2348935,
 2358595,
 2364460,
 2381673,
 2391075,
 2410164,
 2436468,
 2439257,
 2443936,
 2446968,
 2459123,
 2467229,
 2470953,
 2471748,
 2501614,
 2509909,
 2536308,
 2547668,
 2556103,
 2583651,
 2613646,
 2653289,
 2706253,
 2748847,
 2750806,
 2778746,
 2797347,
 2798275,
 2828087,
 2856654,
 2968467,
 2983840,
 2992404,
 3003181,
 3115048,
 3123045,
 3124791,
 3145388,
 3154812,
 3169470,
 3185647,
 3200397,
 3202657,
 3207133,
 3238455,
 3276005,
 3282598,
 3301827,
 3313974,
 3322129,
 3335103,
 3336418,
 3337513,
 3341601,
 3344910,
 3352810,
 3387625,
 3391219,
 3393602,
 3397211,
 3412886,
 3413851,
 3422269,
 3470153,
 3474413,
 3480049,
 3497765,
 3507539,
 3519835,
 3530514,
 3532686,
 3546136,
 3557984,
 3594170,
 3599019,
 3608228,
 3623824,
 3649737,
 3683698,
 3704198,
 3715356,
 3727691,
 3745026,


In [19]:
user_set = set(lowms_users)
statement_events = "SELECT user_id, track_id, artist_id, timestamp FROM events WHERE user_id IN " + str(tuple(user_set))
lowms_events_df = pd.read_sql(con=engine, sql=statement_events)

In [20]:
len(lowms_events_df), len(lowms_events_df.dropna())

(7941793, 7941793)

In [36]:
lowms_events_df = (lowms_events_df[~lowms_events_df["track_id"].isin(dominant_tracks)])

In [25]:
tracks_with_genres_df = pd.read_csv("data/artist_track_genres_prepared.csv", sep=COL_SEPARATOR, header=None)
tracks_with_genres_df.columns = ["track_id", "title", "artist_id", "artist", "playcount", "genres"]
tracks_with_genres_df.drop_duplicates(subset=["track_id", "title"], inplace=True)
tracks_with_genres_df.dropna(inplace=True)
tracks_with_genres_df["genres"] = tracks_with_genres_df.apply(lambda r: _genres_to_list(str(r[-1])), axis=1)
tracks_with_genres_df.head()

C:\Users\pmuellner\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


,track_id,title,artist_id,artist,playcount,genres
0,11393952,Somebody That I Used to Know,3085,Gotye,185187,"[electronic, pop, indiepop, rock, singersongwr..."
1,4868,Rolling in the Deep,504,Adele,171202,"[soul, pop, singersongwriter, blues, jazz, ind..."
2,2900,Intro,32,The xx,130971,"[electronic, indiepop, shoegaze, dreampop, pos..."
3,2498,Blue Jeans,402,Lana Del Rey,130912,"[pop, soul, indiepop, chamberpop, triphop, dre..."
4,26142,We Are Young (feat. Janelle MonÃ¡e),3151,fun.,119257,"[pop, indiepop, rock, alternativerock, alterna..."


In [26]:
tracks_with_genres_df.isna().sum(axis=0)

track_id     0
title        0
artist_id    0
artist       0
playcount    0
genres       0
dtype: int64

In [27]:
(tracks_with_genres_df.dropna())["track_id"].nunique()

320504

In [28]:
tracks_with_genres = tracks_with_genres_df["track_id"].unique()
len(tracks_with_genres)

320504

In [29]:
tracks_with_acoustic_features_df = pd.read_sql(sql="SELECT track_id, danceability, energy, speechiness, acousticness, instrumentalness, tempo, valence, liveness FROM acoustic_features", con=engine)
tracks_with_acoustic_features_df.dropna(inplace=True)
tracks_with_acoustic_features_df.head()

,track_id,danceability,energy,speechiness,acousticness,instrumentalness,tempo,valence,liveness
0,1,0.463,0.877,0.0307,0.000111,0.000006,173.135,0.698,0.523
1,6,0.457,0.387,0.0314,0.728000,0.127000,120.011,0.041,0.114
2,7,0.268,0.903,0.0644,0.025200,0.016200,154.988,0.464,0.206
3,9,0.502,0.980,0.0591,0.000021,0.025600,121.962,0.388,0.102
4,11,0.200,0.275,0.0478,0.030200,0.606000,84.479,0.248,0.850


In [30]:
tracks_with_acoustic_features = tracks_with_acoustic_features_df["track_id"].unique()
len(tracks_with_acoustic_features)

3468289

In [31]:
dominant_tracks = pd.read_csv("../dominant_tracks.csv", header=None)[0].values.tolist()
len(dominant_tracks)

6056

In [37]:
df = pd.read_csv("data/lowms_les.csv", sep=";")

In [38]:
len(df), df["user_id"].nunique(), df["track_id"].nunique(), df["artist_id"].nunique()

(4725664, 2073, 147156, 14316)

In [39]:
len(lowms_events_df)

7836811

In [162]:
statement = "SELECT track_id, danceability, energy, speechiness, acousticness, instrumentalness, tempo, valence, liveness FROM acoustic_features"
df = pd.read_sql(sql=statement, con=engine).set_index("track_id")
print(len(df))
#df = df.loc[rel_tracks].dropna()
print(len(df))


track_genres_df = pd.read_csv("data/track_genres.csv", sep=";")
track_genres_df.columns = ["track_id", "genres"]
track_genres_df = track_genres_df[track_genres_df["genres"] != "[]"]
track_genres_df.set_index("track_id", inplace=True)

# use only tracks with genre annotations
df = df.loc[track_genres_df.index].dropna()
print(len(df))

3471884
3471884
147155


In [163]:
df.reset_index(inplace=True)
df = df[~df["track_id"].isin(dominant_tracks)]

In [164]:
len(df)

145130

In [86]:
track_genres_df = pd.read_csv("data/track_genres.csv", sep=";")
track_genres_df.columns = ["track_id", "genres"]
track_genres_df = track_genres_df[track_genres_df["genres"] != "[]"]
track_genres_df.set_index("track_id", inplace=True)
track_genres_df.head()

,genres
track_id,
2900,"['electronic', 'indiepop', 'shoegaze', 'dreamp..."
572665,"['soul', 'pop', 'singersongwriter', 'blues', '..."
2897,"['indierock', 'electronic', 'indiepop', 'postp..."
15100,"['folk', 'indiefolk', 'banjo', 'folkrock', 'bl..."
7112,"['rock', 'indierock', 'alternativerock', 'pop'..."


In [105]:
len(track_genres_df)

147155

In [15]:
tracks_with_genres_df = tracks_with_genres_df[["track_id", "genres"]]
tracks_with_genres_df.set_index("track_id", inplace=True)
tracks_with_genres_df.head()

,genres
track_id,
11393952,"[electronic, pop, indiepop, rock, singersongwr..."
4868,"[soul, pop, singersongwriter, blues, jazz, ind..."
2900,"[electronic, indiepop, shoegaze, dreampop, pos..."
2498,"[pop, soul, indiepop, chamberpop, triphop, dre..."
26142,"[pop, indiepop, rock, alternativerock, alterna..."


In [ ]:
1+1

In [40]:
lowms_events_df = lowms_events_df[lowms_events_df["track_id"].isin(tracks_with_genres_df["track_id"].unique())]
lowms_events_df["track_id"].nunique()

157488

In [41]:
lowms_events_df.dropna(inplace=True)

In [42]:
len(lowms_events_df)

4917260